# 서론
이 시리즈는 SH 서울주택도시공사에서 발주한 **주거상담데이터 분석을 통한 주거상담소 운영성과 연구용역**을 수행하며 분석한 내용을 바탕으로 합니다. 이 연구용역은 [주거복지상담](https://www.seoulhousing.kr/)을 하며 쌓인 상담기록을 분류하여 주거복지상담의 중요성이나 역할 등을 밝히는 것이 목적이었습니다.
![](image/overview.png)




주거상담소
이 연구용역에서 저는 축적된 주거상담기록 DB를 전처리하고 






# 




**텍스트 전처리**는 주거상담 원형 텍스트를 정제하고 토큰화하는 작업을 말한다. 비정형 텍스트 데이터는 그대로 활용할 수 없어 취득 이후, 전처리하는 과정을 거쳐야 한다.

![](image/overview.png)

### 1. 텍스트 정제

#### 1.1. 패키지 설치 및 데이터 불러오기

이번 작업에서 필요한 패키지를 설치하는 코드는 아래와 같다.


In [ ]:
#| eval: false
## 필요한 패키지 설치하기
!pip install pandas
!pip install tqdm
!pip install kiwipiepy

설치 후, 필요한 패키지를 먼저 불러온다.


In [ ]:
## 필요한 모듈 불러오기
import json, random, re, os
import swifter
import pandas as pd
from tqdm import tqdm # 작업 프로세스 시각화
from kiwipiepy import Kiwi # 형태소분석기 모듈
from kiwipiepy.utils import Stopwords # 불용어사전 담겨있는 모듈 불러오기

csv로 저장된 데이터를 불러온다. 예제 파일은 [여기](data/db_record_v1_sample.csv)에서 다운로드 할 수 있다. `id_f`는 각 데이터 고유 ID, `cons_text`는 형태소 분석 할 상담 내용이다.


In [ ]:
db_record = pd.read_csv("data/db_record_v1_sample.csv") # 데이터 불러오기

In [ ]:
#| echo: false
db_record[0:2].style.hide_index()

#### 1.2. 불필요한 토큰 제거

원형 텍스트에서 **불필요한 토큰**인 기호, 태그, 등을 삭제한다. 때로는 숫자와 영어도 불필요하다고 여겨지지만 '1인가구', '2룸', 'LH', 'SH' 등 의미있는 토큰이 많아 삭제하지 않았다. 아래 코드로 먼저 텍스트 내 영어 표현을 모두 소문자화 한 후, 한글과 숫자, 영어 외 나머지를 공백 처리했다.


In [ ]:
db_record['cons_text_cleaned_1'] = list(map(lambda text: re.sub(r"([^가-힣0-9a-z ])", " ", text), db_record['cons_text'].str.lower()))

In [ ]:
#| echo: false

db_record[0:2][['cons_text', 'cons_text_cleaned_1']].style.hide_index()

#### 1.3. 동의어 사전

**같은 의미지만 다른 표현으로 쓰는 단어**를 통일한다. 아래와 같이 구축한 동의어 사전을 활용하며, 사전은 바꿔줄 단어`beforeWord`, 바꿀 단어`afterWord`로 구성했다.


In [ ]:
#| echo: false
ex_synonyms = pd.read_csv('data/list_ex_synonyms.csv') # 심층상담 데이터 읽기
ex_synonyms.style

'재개발 임대'는 '재개발'과 '임대'가 아닌 '재개발임대'의 다른 표현이므로 이를 통일시켰다. '서울 주거포털'도 마찬가지 사례다. '아산사회복지재단'은 '현대아산재단', '아산재단' 등 다르게 쓰는 표현을 일치시켜주었다.

동의어 사전으로 상담 원문 내용을 바꾸는 코드는 아래와 같다. 함수 `replace_word`는 한 개의 상담 내용`cons_text` 열마다 동의어사전`dict_synonyms`에서 `beforeWord`와 일치하는 구간이 발견되면 `afterWord`로 바꾼다.

#### 1.4. 적용

위 작업을 하는 코드는 아래와 같다.


In [ ]:
## 구축한 동의어사전 불러오기
dict_synonyms = pd.read_csv('data/dict/dict_synonyms.csv')

## 동의어사전을 적용하는 함수 정의
def replace_word(text):
    for i in range(len(dict_synonyms['beforeWord'])): 
        try:
            if dict_synonyms['beforeWord'][i] in text: # beforeWord가 상담내용에서 발견된다면
                text = text.replace(dict_synonyms['beforeWord'][i], dict_synonyms['afterWord'][i]) # 해당하는 단어의 afterWord로 바꾸어라
        except Exception as e: # 에러가 있다면 멈추고 알려주어라
            print(f"Error 발생 / 에러명: {e}")
            print(dict_synonyms['afterWord'][i])
            print(text)
    return text

db_record['cons_text_cleaned_2'] = db_record['cons_text_cleaned_1'].swifter.set_npartitions(npartitions = 12).apply(lambda x: replace_word(x))

In [ ]:
#| echo: false
db_record[0:2][['cons_text', 'cons_text_cleaned_2']].style.hide_index()

### 2. 텍스트 토큰화

#### 2.1. 형태소분석기 Kiwi

다양한 형태소분석기 중 Kiwi를 선택했다. 빠른 속도와 준수한 정확도, 그리고 사용자사전 수정과 활용이 쉽기 때문이다. 앞의 두 장점과 분석기 설명은 [여기](https://lab.bab2min.pe.kr/kiwi)에서 확인 가능하다.

기본 설정으로 Kiwi 분석기로 형태소 분석을 한 결과는 다음과 같다.


In [ ]:
kiwi = Kiwi() # Kiwi 기본값 설정
kiwi.tokenize("서울주거포털에서 재개발임대 정보를 알아보셰요")

형태소화된 토큰은 원형`form`과 형태소`tag`, 시작위치`start`, 길이`len`으로 구성된다.


In [ ]:
#| echo: false

print("[Token(form='서울', tag='NNP', start=0, len=2), \n Token(form='주거', tag='NNG', start=2, len=2), \n Token(form='포털', tag='NNG', start=4, len=2), \n Token(form='에서', tag='JKB', start=6, len=2), \n Token(form='재', tag='XPN', start=9, len=1), \n Token(form='개발', tag='NNG', start=10, len=2), \n Token(form='임대', tag='NNG', start=12, len=2), \n Token(form='장보', tag='NNP', start=15, len=2), \n Token(form='를', tag='JKO', start=17, len=1), \n Token(form='알', tag='VV', start=19, len=1), \n Token(form='어', tag='EC', start=20, len=1), \n Token(form='보', tag='VX', start=21, len=1), \n Token(form='세', tag='EC', start=22, len=1), \n Token(form='요', tag='JX', start=23, len=1)]")

#### 2.2. Kiwi 추가 설정

이 모듈에서 몇가지 추가 설정을 한 코드는 다음과 같다.\
![](image/kiwi_setting.png)

1은 형태소 분석기에 활용하는 모델을 지정하고 2는 기본적인 맞춤법 검사를 하게끔 한다(자세한 정보는 [여기](https://bab2min.github.io/kiwipiepy/v0.14.1/kr/)).

4는 불용어를 걸러내준다. `Stopword()`은 Kiwi에서 제공하는 기본 불용어사전이다.

3은 사용자정의사전을 추가하는 것이다. 위 결과에서 보이듯 사용자정의사전을 추가하지 않으면 '서울주거포털'은 '서울 + 주거 + 포털'로 나눠지게 된다. 사용자정의사전을 만드는 코드는 아래와 같다.


In [ ]:
#| eval: false

## 동의어사전에서 추출하기
dict_synonyms = pd.read_csv('data/dict/dict_synonyms.csv') # 동의어사전 불러오기
list_userDefined_synonyms = (dict_synonyms['afterWord'].str.strip().unique() + "\tNNG\t9").tolist() # 사용자 정의 사전 포맷으로 변경

## 사용자 정의 사전 불러오기
dict_add = pd.read_table("data/dict/dict_userDefined_madeByMe.txt") # 추가 사전 불러오기
list_userDefined_add = (pd.read_table("data/dict/dict_userDefined_madeByMe.txt", header = None, sep = "-")[0] + "\t9").tolist() # 사용자 정의 사전 포맷으로 변경

## 위 두 사전 합치기
dict_userDefined = pd.DataFrame(pd.DataFrame(list_userDefined_synonyms + list_userDefined_add)[0].unique()) # 두 list 결합
dict_userDefined.to_csv('data/dict/dict_userDefined.txt', index = False, header = False) # 새로운 사용자 정의 사전 저장하기

동의어 사전 부분은 `afterWord`를 고유명사화(NNG) 하는 작업이다. 동의어 사전에서 지정한 단어들은 '1인가구', '미성년자', '늘푸른돌봄센터' 등으로 대부분 명사에 속하기 때문이다.

따로 만든 사용자 정의 사전에는 명사 외 사용자 정의를 해줄 단어가 들어있다. '통보함'이 '통보함(NNG/명사)'가 아니라 '통보/NNG + 하/VV + 다/EF', '연락키로'가 '연락키로(NNG/명사)'가 아니라 '연락/NNG + 하/XSV + 기/ETN + 로/JKB'로 형태소화 되게끔 만들었다.

추가 설정을 한 형태소 분석의 결과는 다음과 같다.


In [ ]:
#| eval: false

## Kiwi 설정
kiwi = Kiwi(model_type = 'sbg', typos = 'basic') # Kiwi 추가 설정
kiwi.load_user_dictionary('data/dict/dict_userDefined.txt') # 사용자사전 추가
kiwi.tokenize("서울주거포털에서 재개발임대 정보를 알아보셰요", stopwords = Stopwords())

In [ ]:
#| echo: false

print("[Token(form='서울주거포털', tag='NNG', start=0, len=6), \n Token(form='재개발임대', tag='NNG', start=9, len=5), \n Token(form='정보', tag='NNG', start=15, len=2), \n Token(form='알', tag='VV', start=19, len=1), \n Token(form='보', tag='VX', start=21, len=1), \n Token(form='세', tag='EC', start=22, len=1), \n Token(form='요', tag='JX', start=23, len=1)]")

#### 2.3. 적용

위 작업을 적용한 결과는 다음과 같다.


In [ ]:
## Kiwi 설정
kiwi = Kiwi(model_type = 'sbg', typos = 'basic') # Kiwi 추가 설정
kiwi.load_user_dictionary('data/dict/dict_userDefined.txt') # 사용자사전 추가
morph_analysis = lambda x: kiwi.tokenize(x, stopwords = Stopwords()) if type(x) is str else None # 형태소 분석
db_record['cons_text_posTagged_kiwi'] = db_record['cons_text_cleaned_2'].swifter.apply(morph_analysis)

In [ ]:
#| echo: false

db_record[0:2][['cons_text', 'cons_text_posTagged_kiwi']].style.hide_index()

### 3. 불용어 제거 및 형태소 결합

#### 3.1. 불용어 사전

kiwi 기본 불용어 사전에서 거르지 못한 불용어를 거르기 위해 사전을 만든다. '대상자'와 '서비스', '주거' 등은 자주 등장하지만 크게 의미있는 단어가 아니다. 지역명과 상담사명도 마찬가지다.


In [ ]:
#| echo: false

ex_synonyms = pd.read_csv('data/list_ex_stopword.csv') # 심층상담 데이터 읽기
ex_synonyms.style

#### 3.2. 한글자 사전

단어가 한글자지만 중요한 것을 모아둔 사전이다. 보통 한글자는 의미가 잘 없다. 하지만 '돈'은 경제적 상황을 서술할 때 사용하며, '좁'은 '좁다', '좁은 집'으로, '낡'은 '낡다' 등으로 활용된다. 이 같은 의미있는 한글자는 불용어로 걸러지지 않게끔 한다. 형태소 분석한 토큰이 아주 짧은 경우는 보통 제거한다.


In [ ]:
#| echo: false

ex_synonyms = pd.read_csv('data/list_ex_oneChar.csv') # 심층상담 데이터 읽기
ex_synonyms.style

#### 3.3. 적용

위 두 사전과 함께 형태소화된 토큰을 결합하는 단계는 아래와 같다.


In [ ]:
## 형태소분석된 것들을 join
dict_stopword = pd.read_csv('data/dict/dict_stopword.csv')
dict_oneChar = pd.read_csv("data/dict/dict_oneChar.csv")

## 품사 정의하기
list_pos_all = ("N", "V", "M", "J", "E", "X") # 주요 품사 모두
list_pos_main = ("N", "V") # 명사, 동사, 형용사만

## 불용어 제거하고 형태소 결합하기
def tokenToText(tokens):
    textJoinedMain = []
    textJoinedAll = [] 
    for token, pos, _, _ in tokens:        
        if (token not in list(dict_stopword['word']) and ( len(token) > 1 or (len(token) == 1 and token in list(dict_oneChar['word']) ) )): 
            if pos.startswith('V'): token = token + '다' # 동사라면 뒤에 '다'를 붙여서 자연스럽게 만들기
            if pos.startswith(list_pos_main) : textJoinedMain.append(token)
            if pos.startswith(list_pos_all) : textJoinedAll.append(token)

    out = pd.Series([' '.join(textJoinedAll), ' '.join(textJoinedMain)])
    return out

db_record[['cons_text_posTagged_all', 'cons_text_posTagged_main']] = db_record['cons_text_posTagged_kiwi'].swifter.set_npartitions(npartitions = 6).apply(lambda x: tokenToText(x))

In [ ]:
#| echo: false

db_record[0:2][['cons_text', 'cons_text_posTagged_main']].style.hide_index()

### 끝으로

위 과정을 종합한 코드는 [여기](data/2.%20morphAnalysis.ipynb)에서 다운로드 가능하다.